## Setup and login
- Define the database backend. This is the default.
- Inititize FetchJson
- login, Make sure to have a config.ini configured

In [ ]:
from zpdatareader import FetchJson, ZDatabase
# Define the database
db = ZDatabase(db_path="database/z_database.json")
z = FetchJson(db=db)
z.login()

## Get event result and plot
- All API reusts are cached into local TinyDB database by default.
- The api data is not proccessed.
- If multiple APIs are present they are combined

In [ ]:
result = z.fetch_result(zid=2552316)
print(f"Event ID, 'zid' is: {result['zid']}")
print(f"Top level data in JSON: {result.keys()}")
print("Top five")
for racer in result['zwift_data'][:5]:
    if int(racer['pos']) <=5:
        print(f"{racer['pos']}: {racer['name']} with a time of {racer['race_time'][0]}")

##Getting started with analysis
- Tools for this will be added in the future
- Many columns are a list of two values with the second=0 use splitlist()
- Many integer columns have blank "" values that mayb better be 0

In [ ]:
import pandas as pd
df = pd.DataFrame(result['zwift_data'])

def splitlist(df, col, drop2=True):
    df[[f'{col}', f'{col}_2' ]] = df[col].tolist()
    if drop2:
        df.drop(f'{col}_2', axis=1, inplace=True)

splitlist(df, 'watts')
splitlist(df, 'wkg')
splitlist(df, 'wkg_ftp')

df.head()

In [ ]:
df[['watts', 'wkg', 'wkg_ftp']] = df[['watts', 'wkg', 'wkg_ftp']].astype(float)

In [ ]:
df[['wkg', 'wkg_ftp']].plot()